In [1]:
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt

Problem 1

In [2]:
# m_cluster = 250
# gamma = 0.2
# epsilon = 10**(-3)
# delta = 0.01
# k_cluster = 10
# m = 250
# r = 0.95
# theta = np.pi/4
# k_max = 250

# def objective_function(x):
#     f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
#     f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
#     f_list = [f1,f2]
#     denom = 0
#     for f in f_list:
#         denom +=np.abs(f)
#     F = 1/(1+denom)
#     return F

# boundaries = np.array([(-10,10),(-10,10)])

Problem 2

In [3]:
# m_cluster = 2000
# gamma = 0.3
# epsilon = 10**(-3)
# delta = 0.1
# k_cluster = 10
# m = 300
# r = 0.95
# theta = np.pi/4
# k_max = 300

# def objective_function(x):
#     f1 = 0.5*np.sin(x[0]*x[1])-0.25*x[1]/np.pi - 0.5*x[0]
#     f2 = (1-0.25/np.pi)*(np.exp(2*x[0])-np.exp(1))+np.exp(1)*x[1]/np.pi - 2*np.exp(1)*x[0]
#     f_list = [f1,f2]
#     denom = 0
#     for f in f_list:
#         denom +=np.abs(f)
#     F = 1/(1+denom)
#     return F

# boundaries = np.array([(-1,3),(-17,4)])

Problem 5

In [4]:
m_cluster = 3000
gamma = 0.1
epsilon = 5*10**(-3)
delta = 0.01
k_cluster = 10
m = 200
r = 0.95
theta = np.pi/4
k_max = 200

def objective_function(x):
    f1 = 2*x[0]+x[1]+x[2]+x[3]+x[4]-6
    f2 = x[0]+2*x[1]+x[2]+x[3]+x[4]-6
    f3 = x[0]+x[1]+2*x[2]+x[3]+x[4]-6
    f4 = x[0]+x[1]+x[2]+2*x[3]+x[4]-6
    f5 = x[0]*x[1]*x[2]*x[3]*x[4]-1
    f_list = [f1,f2,f3,f4,f5]
    denom = 0
    for f in f_list:
        denom +=np.abs(f)
    F = 1/(1+denom)
    return F

boundaries = np.array([(-10,10) for _ in range (5)])

In [5]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points

"""MAXIMIZE FUNCTION"""
def maximize(set_of_points):
    z = []
    z_max = 0
    F = objective_function(set_of_points.T)
    for i in range (len(set_of_points)):
        z.append(F[i])
        if z[i]>z_max:
            z_max = z[i]
            idx_max = i
    x_max = set_of_points[idx_max]
    return z_max,idx_max,x_max


In [6]:
k=0
iter_points = {}
iter_points[k] = generate_points(len(boundaries),m_cluster,boundaries[:,0],boundaries[:,1])
x_prime = maximize(iter_points[0])[-1]
x_prime

array([ 1.58203125, -0.25390625, -1.23046875,  0.09765625,  5.13671875])

In [7]:
min_boundaries = 10**100
for i in range (len(boundaries)):
    abs_disc = np.abs(boundaries[i,1]-boundaries[i,0])
    if abs_disc<=min_boundaries:
        min_boundaries = abs_disc
radius = min_boundaries/2
radius

10.0

In [8]:
k = 0
cluster_center,cluster_radius = {},{}
cluster_center[0],cluster_radius[0] = x_prime,radius
print(cluster_center,cluster_radius)

{0: array([ 1.58203125, -0.25390625, -1.23046875,  0.09765625,  5.13671875])} {0: 10.0}


In [9]:
# potential_cluster_center = []
# F = objective_function(iter_points[0].T)
# for i in range (m_cluster):
#     if F[i] > gamma:
#         potential_cluster_center.append(iter_points[0][i])
# len(potential_cluster_center)

In [10]:
"""FUNCTION CLUSTER"""
def function_cluster(y,lendict):
    min_dist_cluster = 10**100
    for ci,cc in cluster_center.items():
        dist = np.linalg.norm(cc-y)
        if dist<=min_dist_cluster:
            xc = cc
            cluster_id = ci
            min_dist_cluster = dist
    xt = (xc + y)/2
    # print(xt,xc,y)
    Fxt = objective_function(xt)
    Fxc = objective_function(xc)
    Fy = objective_function(y)
    # print(Fxt,Fxc,Fy)
    if Fxt < Fy and Fxt < Fxc:
        cluster_center[lendict] = y
        cluster_radius[lendict] = np.linalg.norm(y-xt)
    elif Fxt > Fy and Fxt > Fxc:
        cluster_center[lendict] = y
        cluster_radius[lendict] = np.linalg.norm(y-xt)
        function_cluster(xt,lendict+1)
    elif Fy > Fxc:
        cluster_center[cluster_id] = y

    cluster_radius[cluster_id] =  np.linalg.norm(y-xt)
    # return cluster_center,cluster_radius


In [11]:
# for i in range (len(potential_cluster_center)):
#     print(f'Titik ke-{i}')
#     lendict = len(list(cluster_center.keys()))
#     function_cluster(potential_cluster_center[i],lendict)
#     print(cluster_center,cluster_radius)
#     print('')

In [12]:
import spiralopt as spo

Sn = r*spo.generate_Rn(dim=2,theta=theta)
k=0

while k<k_cluster:
    potential_cluster_center = []
    F = objective_function(iter_points[k].T)
    for i in range (m_cluster):
        if F[i] > gamma:
            potential_cluster_center.append(iter_points[k][i])
    len(potential_cluster_center)
    # print('')
    for i in range (len(potential_cluster_center)):
        # print(f'Titik ke-{i}')
        lendict = len(list(cluster_center.keys()))
        function_cluster(potential_cluster_center[i],lendict)
        # print(cluster_center,cluster_radius)
        # print('')
    iter_points[k+1]=spo.update_point(iter_points[k],Sn,dim=len(boundaries),objective_function=objective_function)
    k+=1
cluster_center,cluster_radius

ValueError: shapes (2,2) and (5,1) not aligned: 2 (dim 1) != 5 (dim 0)

In [ ]:
def cluster_boundaries(center_point,radius):
    return np.array([[center_point[i]-radius,center_point[i]+radius]for i in range (len(center_point))])

In [ ]:
roots = []
roots_values = []
for i in range (len(cluster_center)):
    bound = cluster_boundaries(cluster_center[i],cluster_radius[i])
    lp = bound[:,0]
    hp = bound[:,1]
    root,value = spo.SpiralOpt(lp,hp,objective_function,dim=2,npoint=m,r = r,theta=theta, iter_max=k_max, error_max = 10**(-5),random=0, show_err=False, show_objective_function=False)
    roots.append(root)
    roots_values.append(value)
print(roots)
print(roots_values)

[[0.5000022717735871, 3.141597096059079], [-0.26059639105515375, 0.6225304875959872], [1.174236251433537, -0.8675164933547067], [-0.24055652850290282, 0.7226292274218346], [1.0890859864631193, 0.3413046900030345], [-0.4978567377049285, -0.6295990042143522], [1.5460004774594744, -10.819768505056128], [-0.15647265582944417, 1.1305175882418348], [1.672637529162629, -16.77604437203473], [1.6499272038048727, -15.578693459030585], [-0.31740702746976, 0.4306882944740895], [-0.941206787091725, -3.1840871923480205], [1.529879433956135, -10.177714863008227], [-0.5025568152106246, -0.6554115524526947], [0.011345737350102879, 1.8747279523622113], [0.9232301028434668, 1.9510185455417484], [1.4036273594086066, -5.909676047944996], [1.3051610424056375, -3.378590379317089], [-0.4391014358140238, -0.3093901171914397], [1.5693863178461869, -11.793985606760712], [0.500001041540945, 3.1415903335002686], [-0.28247683134266605, 0.5121106311248332], [1.4339490480199284, -6.82076345353203], [0.028564902642815

In [ ]:
eligible_roots = np.array([x for x in roots if (1-objective_function(x))<epsilon])
eligible_roots

array([[  0.50000227,   3.1415971 ],
       [ -0.26059639,   0.62253049],
       [  0.50000104,   3.14159033],
       [  1.43394905,  -6.82076345],
       [  1.57822585, -12.17669252],
       [  0.29944628,   2.83692242],
       [  0.29839573,   2.8343849 ],
       [  0.30129464,   2.8413644 ],
       [  0.29846173,   2.83454126],
       [  0.29944976,   2.83693031],
       [  0.30074915,   2.8400567 ],
       [  0.29749095,   2.83219329],
       [  0.29846257,   2.83454591],
       [  0.29911608,   2.83612537],
       [  0.30069857,   2.83993582],
       [  0.30054002,   2.83955434],
       [  0.30078112,   2.84014218],
       [  0.29893042,   2.83567863],
       [  0.30038751,   2.83918987],
       [  0.29944714,   2.83692285],
       [  0.2995105 ,   2.83714902],
       [  0.29934454,   2.83670692],
       [  0.30015411,   2.83862885],
       [  0.29944715,   2.83692486],
       [  0.30000601,   2.83826808],
       [  0.30021978,   2.83878479],
       [  0.29936216,   2.83671902],
 

In [ ]:
duplicated_roots = []
for i in range(len(eligible_roots)):
    for j in range (i+1,len(eligible_roots)):
        if np.linalg.norm(eligible_roots[i]-eligible_roots[j])<delta:
            duplicated_roots.append([eligible_roots[i],eligible_roots[j]])
duplicated_roots = np.unique(duplicated_roots,axis=0)
# print(duplicated_roots)
deselected_duplicated_roots = []
for i in range (len(duplicated_roots)):
    root_a = objective_function(duplicated_roots[i][0])
    root_b = objective_function(duplicated_roots[i][1])
    if root_a>root_b:
        deselected_duplicated_roots.append(list(duplicated_roots[i][1]))
    else:
        deselected_duplicated_roots.append(list(duplicated_roots[i][0]))
# print(deselected_duplicated_roots)

if deselected_duplicated_roots:
    exclude_condition = np.all(eligible_roots != np.array(deselected_duplicated_roots)[:, np.newaxis], axis=2).all(axis=0)
    final_root = eligible_roots[exclude_condition]
else:
    final_root = eligible_roots
print(final_root)

[[ -0.26059639   0.62253049]
 [  0.50000104   3.14159033]
 [  1.43394905  -6.82076345]
 [  1.57822585 -12.17669252]
 [  0.29944976   2.83693031]]


In [ ]:
# """Visualization"""
# fig, ax = plt.subplots()
# for center,radius in zip(cluster_center.values(),cluster_radius.values()):
#     circle = plt.Circle(center, radius, fill=False, linestyle='dotted', edgecolor='b')
#     ax.add_artist(circle)

# # Set axis limits
# ax.set_xlim(-10, 10)
# ax.set_ylim(-10, 10)

# # # Add labels (optional)
# # for i, center in cluster_center.items():
# #     ax.text(center[0], center[1], f'Cluster {i+1}', ha='center', va='bottom')

# # Add a title and labels (optional)
# ax.set_title('Cluster Visualization')
# ax.set_xlabel('X-axis')
# ax.set_ylabel('Y-axis')

# # Show the plot
# plt.gca().set_aspect('equal', adjustable='box')  # Make the aspect ratio equal
# plt.grid(True)
# plt.show()
